In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint

from ads_query_eval.config import get_terminus_client, get_terminus_config
from ads_query_eval.app.bootstrap import QUERIES

client = get_terminus_client()

Checking for 200 response from http://terminus:6363/api/info...


In [3]:
for name, defn in client.get_existing_classes().items():
    print(name)
    pprint(defn)
    print()

JobCategory
{'@id': 'JobCategory',
 '@type': 'Enum',
 '@value': ['ScheduledRetrieval',
            'SensedEvaluationOfRetrievedItemsByProcedure']}

Relevance
{'@id': 'Relevance', '@type': 'Enum', '@value': ['not relevant', 'relevant']}

JobRun
{'@documentation': {'@comment': 'a run of a job, i.e. a run of a configured '
                                'graph of operations',
                    '@properties': {'category': 'the category of job',
                                    'operations': 'the set of operations for '
                                                  'this job'}},
 '@id': 'JobRun',
 '@type': 'Class',
 'category': 'JobCategory',
 'operations': {'@class': 'Operation', '@type': 'Set'}}

Uncertainty
{'@id': 'Uncertainty',
 '@type': 'Enum',
 '@value': ['not supplied', 'low', 'high']}

EvaluatingProcedure
{'@documentation': {'@comment': 'an automated evaluator',
                    '@properties': {'config': 'Needed configuration for a '
                                   

In [4]:
print(len(client.get_all_documents(as_list=True)), "documents")
print(
    len(client.get_documents_by_type("Query", as_list=True)),
    "query documents"
)
print(
    len(client.get_documents_by_type("EvaluatingProcedure", as_list=True)),
    "evaluating-procedure documents"
)    

15 documents
14 query documents
1 evaluating-procedure documents


In [5]:
client.get_documents_by_type("Query", as_list=True)

[{'@id': 'Query/full%3A%22coronal%20mass%20ejection%22',
  '@type': 'Query',
  'query_literal': 'full:"coronal mass ejection"',
  'query_literal_md5': '4d87af6b3ff135f826c327a5cf93b7d2'},
 {'@id': 'Query/full%3A%22geomagnetically%20induced%20current%22',
  '@type': 'Query',
  'query_literal': 'full:"geomagnetically induced current"',
  'query_literal_md5': '04c482b968f43ecf35de3effe96a5843'},
 {'@id': 'Query/full%3A%22ionospheric_conductivity%22',
  '@type': 'Query',
  'query_literal': 'full:"ionospheric_conductivity"',
  'query_literal_md5': '7efaf0c9f9d6751791c22539f25ada4d'},
 {'@id': 'Query/full%3A%22particle%20acceleration%22',
  '@type': 'Query',
  'query_literal': 'full:"particle acceleration"',
  'query_literal_md5': '04a9255fa358944f912873c16fa59941'},
 {'@id': 'Query/full%3A%22solar%20wind%22',
  '@type': 'Query',
  'query_literal': 'full:"solar wind"',
  'query_literal_md5': '4b19ebf3a83a43721c61e9984d0f0671'},
 {'@id': 'Query/full%3A%22space%20weather%22',
  '@type': 'Query

In [6]:
client.get_documents_by_type("EvaluatingProcedure", as_list=True)

[{'@id': 'EvaluatingProcedure/ads_query_eval.frame.evaluators.topic_review_references+0.1',
  '@type': 'EvaluatingProcedure',
  'config': {'full:"coronal mass ejection"': ['bibcode:2017LRSP...14....5K',
    'bibcode:2012LRSP....9....3W'],
   'full:"geomagnetically induced current"': ['bibcode:2017SpWea..15..828P',
    'bibcode:2017SpWea..15..258B'],
   'full:"ionospheric_conductivity"': ['bibcode:1993JATP...55.1493B',
    'bibcode:2008AnGeo..26.3913A',
    'bibcode:2012GMS...197..143M',
    'bibcode:1956NCim....4S1385C'],
   'full:"particle acceleration"': ['bibcode:2012SSRv..173..433F',
    'bibcode:2012SSRv..173..103M'],
   'full:"solar wind"': ['bibcode:2021LRSP...18....3V'],
   'full:"space weather"': ['bibcode:2006LRSP....3....2S',
    'bibcode:2021LRSP...18....4T',
    'bibcode:2007LRSP....4....1P',
    'bibcode:2015AdSpR..55.2745S',
    'bibcode:2022FrASS...8..253B'],
   'full:"substorm"': ['bibcode:2015SSRv..190....1K'],
   'full:("interplanetary magnetic field" AND reconnectio

In [7]:
import requests

def tget(url):
    client._check_connection()
    return requests.get(
        url,
        headers=client._default_headers,
        auth=client._auth(),
    )

def tget_ttl(dbid, which=None):
    assert which in {"schema","instance"}
    rv = tget(f"http://terminus:6363/api/triples/admin/{dbid}/local/branch/main/{which}?format=turtle")
    for line in rv.json().split('\n'):
        print(line)
    return rv

#tget_ttl(client.db, which="schema")
#tget_ttl(client.db, which="instance")